In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# 데이터 생성 함수
def generate_data(num_samples):
    X = np.random.randint(0, 10, (num_samples, 5))
    y = np.flip(X, axis = 1)
    return X, y

In [3]:
# 데이터 생성
num_samples = 20000
X, y = generate_data(num_samples)

In [4]:
# 데이터 형태 변환 (LSTM을 사용하기 위해 3D 텐서로 변환)
X = X.reshape((X.shape[0], X.shape[1], 1))
y = y.reshape((y.shape[0], y.shape[1], 1))

In [5]:
# 모델 생성
model = Sequential()
model.add(LSTM(128, input_shape = (5, 1)))
model.add(Dropout(0.2))
model.add(RepeatVector(5))
model.add(LSTM(128, return_sequences = True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(10, activation = 'softmax')))

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [8]:
# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)

In [9]:
# 모델 학습
model.fit(X, y, epochs = 50, validation_split = 0.2, callbacks = [early_stopping])

Epoch 1/50
500/500 [==============================] - 12s 15ms/step - loss: 1.9717 - accuracy: 0.2299 - val_loss: 1.6208 - val_accuracy: 0.3573
Epoch 2/50
500/500 [==============================] - 7s 15ms/step - loss: 1.1699 - accuracy: 0.5290 - val_loss: 0.6486 - val_accuracy: 0.8806
Epoch 3/50
500/500 [==============================] - 6s 13ms/step - loss: 0.6405 - accuracy: 0.7735 - val_loss: 0.3735 - val_accuracy: 0.9714
Epoch 4/50
500/500 [==============================] - 7s 13ms/step - loss: 0.4770 - accuracy: 0.8369 - val_loss: 0.2578 - val_accuracy: 0.9768
Epoch 5/50
500/500 [==============================] - 7s 14ms/step - loss: 0.4048 - accuracy: 0.8584 - val_loss: 0.1833 - val_accuracy: 0.9908
Epoch 6/50
500/500 [==============================] - 6s 13ms/step - loss: 0.3419 - accuracy: 0.8839 - val_loss: 0.1505 - val_accuracy: 0.9863
Epoch 7/50
500/500 [==============================] - 5s 11ms/step - loss: 0.3106 - accuracy: 0.8936 - val_loss: 0.1491 - val_accuracy: 0.978

In [10]:
# 모델 검증 함수
def predict_reverse(input_sequence):
    input_sequence = np.array(input_sequence).reshape((1, 5, 1))
    predicted_sequence = model.predict(input_sequence)
    return np.argmax(predicted_sequence, axis = -1).reshape((5, ))

In [11]:
# 입력 반복 루프
while True:
    user_input = input("입력 (쉼표로 구분된 5개의 숫자, 종료하려면 'quit' 입력): ")
    
    if user_input.lower() == "quit":
        break
    try:
        input_sequence = list(map(int, user_input.split(',')))
        if(len(input_sequence) != 5):
            print("5개의 숫자를 입력해 주세요.")
            continue
        predicted_output = predict_reverse(input_sequence)
        print(f"입력: {input_sequence}")
        print(f"예측된 출력: {predicted_output}")
    except ValueError:
        print("유효한 숫자를 입력해주세요.")

1/1 [==============================] - 1s 1s/step
입력: [1, 5, 7, 8, 9]
예측된 출력: [9 8 7 5 1]
1/1 [==============================] - 0s 35ms/step
입력: [1, 2, 3, 4, 5]
예측된 출력: [5 4 3 2 1]


In [12]:
# 20개의 data를 만들어 검증
test_X, test_y = generate_data(20)

In [13]:
# 모델 예측 및 정확도 계산
correct_predictions = 0
total_predictions = len(test_X)

for i in range(total_predictions):
    input_sequence = test_X[i]
    true_output = test_y[i].reshape(5, )
    predicted_output = predict_reverse(input_sequence)

    if np.array_equal(predicted_output, true_output):
        correct_predictions += 1

    print(f"입력: {input_sequence.reshape(5, )}")
    print(f"예측된 출력: {predicted_output}")
    print(f"실제 출력: {true_output}")
    print(f"정확 여부: {'맞음' if np.array_equal(predicted_output, true_output) else '틀림'}\n")

accuracy = correct_predictions / total_predictions
print(f"총 정확도: {accuracy * 100:.2f}%")

1/1 [==============================] - 0s 27ms/step
입력: [3 1 7 1 1]
예측된 출력: [1 1 7 1 3]
실제 출력: [1 1 7 1 3]
정확 여부: 맞음

1/1 [==============================] - 0s 35ms/step
입력: [2 3 8 6 0]
예측된 출력: [0 6 8 3 2]
실제 출력: [0 6 8 3 2]
정확 여부: 맞음

1/1 [==============================] - 0s 37ms/step
입력: [8 3 6 8 2]
예측된 출력: [2 8 6 3 8]
실제 출력: [2 8 6 3 8]
정확 여부: 맞음

1/1 [==============================] - 0s 24ms/step
입력: [6 6 3 9 9]
예측된 출력: [9 9 3 6 6]
실제 출력: [9 9 3 6 6]
정확 여부: 맞음

1/1 [==============================] - 0s 42ms/step
입력: [4 9 2 4 1]
예측된 출력: [1 4 2 9 4]
실제 출력: [1 4 2 9 4]
정확 여부: 맞음

1/1 [==============================] - 0s 47ms/step
입력: [4 3 4 2 8]
예측된 출력: [8 2 4 3 4]
실제 출력: [8 2 4 3 4]
정확 여부: 맞음

1/1 [==============================] - 0s 34ms/step
입력: [0 9 3 2 8]
예측된 출력: [8 2 3 9 0]
실제 출력: [8 2 3 9 0]
정확 여부: 맞음

1/1 [==============================] - 0s 33ms/step
입력: [9 1 3 6 1]
예측된 출력: [1 6 3 1 9]
실제 출력: [1 6 3 1 9]
정확 여부: 맞음

1/1 [==============================] - 0s 31ms/step
입력: 